In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys, os

# add relevant folders to sys path (only needed for jupyter)
proj_path = Path().cwd().parent 

try:
    for folder in next(os.walk(proj_path))[1]:
        if '.' not in folder:  # ignore hidden folders like .git
            path_to_add = os.path.join(proj_path, folder)
            sys.path.append(path_to_add)
except StopIteration:
    print(f"No directories found in {proj_path}")

from utils.transaction_data_utils import AugmentTransactionData

In [ ]:
# Initialize class and load data
atd = AugmentTransactionData() 

# get transactions with identifiers (SEPA, BEA, ideal etc..)
transactions = atd.get_transaction_details() 

# # remove sensitive info or columns
transactions = atd.remove_sensitive_data(transactions) # remove sensitive info or columns

transactions.head(                  )


In [ ]:
#  find most frequent transaction types
most_freq_transaction_types = atd.get_most_freq_transaction_types()

# get dataset with company labels
tr_data_with_companies = atd.get_transaction_details(transactions)

# show results
tr_data_with_companies.head()

In [ ]:
# example to show what would happen if wrong data types were used for functions (check the key error)

# lets try using data without identifiers to get companies!
transactions = atd.get_transactions_with_identifiers()

# remove sensitive info
transactions= atd.remove_sensitive_data(transactions)

# try getting companies
tr_data_with_companies = atd.get_transaction_details(transactions)
tr_data_with_companies.head()   
